In [351]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import plotly.graph_objects as go
import networkx as nx

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import image
import nltk
nltk.download('stopwords')
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors
from keras.layers import Embedding
from keras.models import Sequential
import tensorflow as tf
from mtcnn import MTCNN

from tensorflow.keras.layers import Input, Dropout, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Model, load_model
import cv2
import os

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader, Loader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from spektral.utils import normalized_laplacian

import os
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gsevr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [352]:
df = pd.read_csv('MELD.Raw/dev_sent_emo.csv',encoding='utf-8')
df['Utterance'] = df.Utterance.str.replace('',"'")
df['gender'] = df.Speaker.apply(lambda x: 'male' if x == 'Ross' or x == 'Joey' or x == 'Chandler' else 'female')
df = df.sample(n=50,ignore_index=True)
df

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime,gender
0,186,"Yeah, and y'know what? I don't give a tiny rat...",Phoebe,anger,negative,16,12,7,5,"00:20:30,187","00:20:33,397",female
1,964,It's breast milk.,Ross,neutral,neutral,91,1,2,2,"00:09:12,593","00:09:14,928",male
2,238,You wanna explain that?,Ross,neutral,neutral,19,17,1,23,"00:15:03,986","00:15:05,570",male
3,37,"I'm sorry, who are you?",Woman,surprise,positive,4,9,7,7,"00:15:24,798","00:15:27,967",female
4,645,"Ooh, actually I've been making a list of all t...",Carol,joy,positive,62,1,6,16,"00:00:59,309","00:01:03,020",female
5,268,What?,Joey,neutral,neutral,22,9,1,4,"00:18:03,624","00:18:05,041",male
6,670,That's all I'm askin',Ross,neutral,neutral,63,20,8,18,"0:21:01,754","0:21:04,265",male
7,659,Are you kidding?!,Rachel,surprise,positive,63,9,8,18,"0:20:07,527","0:20:08,278",female
8,799,"Okay, y'know what? Can I, can I talk now?",Rachel,joy,positive,74,17,8,3,"0:10:38,604","0:10:40,648",female
9,176,Oh just say it Kyle!,Ross,surprise,negative,16,2,7,5,"00:19:58,030","00:19:59,697",male


In [353]:
def preprocess(text):
    t = text.lower()
    t = re.sub('\d+',r'',t)
    t = re.sub(r'\W+',r' ',t)
    return t

lemmatizer = WordNetLemmatizer()



df['prepro'] = [' '.join([lemmatizer.lemmatize(preprocess(txt))])
                 .strip() for txt in df['Utterance']]


texts = df.prepro.values
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

In [354]:
glove_path = 'glove.6B.50d.txt'
word_vectors = KeyedVectors.load_word2vec_format(glove_path, binary=False, encoding='utf8',no_header=True)



In [355]:
embedding_dim = 50  
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, embedding_dim))

for word, i in tokenizer.word_index.items():
    if word in word_vectors:
        embedding_matrix[i] = word_vectors[word]

embedding_layer = Embedding(
    input_dim=len(tokenizer.word_index) + 1,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=50,
    trainable=False
)


model = Sequential()
model.add(embedding_layer)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics='accuracy')


In [356]:
embedding_vectors = model.predict(padded_sequences)
df['text'] = list(embedding_vectors)

2/2 [==============================] - 0s 5ms/step


In [391]:
facenet_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_38 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_38[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [392]:
detector = MTCNN()

facenet_model = tf.keras.applications.VGG16(weights='imagenet')

def preprocess_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0 
    return img

def extract_face_embeddings(frame):
    faces = detector.detect_faces(frame)
    face_embeddings = []
    for face in faces:
        x, y, w, h = face['box']
        x1, y1 = max(x, 0), max(y, 0)
        x2, y2 = min(x + w, frame.shape[1]), min(y + h, frame.shape[0])
        cropped_face = frame[y1:y2, x1:x2]
        
        # Preprocess face
        preprocessed_face = preprocess_image(cropped_face)
        preprocessed_face = np.expand_dims(preprocessed_face, axis=0)
        
        # Get face embeddings 
        face_embedding = facenet_model.predict(preprocessed_face)
        face_embeddings.append(face_embedding)
    return face_embeddings

# Read video
folder_path = 'MELD.Raw/dev_splits_complete/'
one_face_videos = {}

for idx,row in df.iterrows():
    file_name = 'dia' + str(row['Dialogue_ID']) + '_utt' + str(row['Utterance_ID']) + '.mp4'
    video_path = folder_path + file_name
    if os.path.isfile(video_path):
        video_capture = cv2.VideoCapture(video_path)  # Provide the path to your input video

        single_video_embeddings = []  # List to store embeddings for all frames

        while True:
            ret, frame = video_capture.read()
            if not ret:
                break

            #check for more than 1 faces
            if len(detector.detect_faces(frame)) == 1:
            
                # Extract face embeddings from each frame
                extracted_embeddings = extract_face_embeddings(frame)
                
                single_video_embeddings.append(extracted_embeddings)  # Append embeddings for this frame
                
                # Display the frame with bounding boxes around detected faces
                for face in detector.detect_faces(frame):
                    x, y, w, h = face['box']
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
                cv2.imshow('Video', frame)
                
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

                video_capture.release()
                cv2.destroyAllWindows()

            else:
                break
        if single_video_embeddings != []:            
            one_face_videos[file_name] = single_video_embeddings
        else:
            one_face_videos[file_name] = 'too many faces'

                




1/1 [==============================] - 0s 44ms/step


In [393]:
df['faces'] = one_face_videos.values()

df = df[df.faces != 'too many faces']

df['faces'] = df.faces.apply(lambda x: x[0][0][0])


C:\Users\gsevr\AppData\Local\Temp\ipykernel_14060\1239130037.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [394]:
G = nx.Graph()


In [395]:

emotion_col = {
            'sadness':'teal',
            'surprise':'silver',
            'neutral':'lightgreen',
            'joy':'purple',
            'anger':'black',
            'disgust': 'brown',
            'fear':'yellow'
}

gender_col = {
    'male':'blue',
    'female':'pink'
}

In [396]:
df.faces.iloc[0].shape

(1000,)

In [397]:
for emotion in df.Emotion.unique():
    G.add_node(emotion, color=emotion_col[emotion],hover=emotion,emotion=emotion, type='emotion',text_embedding=np.zeros((50,50)),visual_embedding=np.zeros((1000,)))


In [398]:
for gender in df.gender.unique():
    G.add_node(gender,color=gender_col[gender], hover=gender, gender=gender, type='gender',text_embedding=np.zeros((50,50)),visual_embedding=np.zeros((1000,)))

In [399]:
for idx, row in df.iterrows():
    G.add_node(row['Utterance'], 
    text_embedding=row['text'],
    visual_embedding=row['faces'],
    color=emotion_col[row['Emotion']],
    hover=row['Utterance'],
    emotion=row['Emotion'],
    gender=row['gender'], 
    type='utterance',
    padding='zero')

In [400]:
for idx, row in df.iterrows():
    G.add_edge(row['Utterance'], row['Emotion'],color=emotion_col[row['Emotion']],weight=1)
    G.add_edge(row['Utterance'], row['gender'], color=gender_col[row['gender']],weight=1)


In [401]:
pos = nx.spring_layout(G)
# pos = nx.fruchterman_reingold_layout(G)


In [402]:
import textwrap

fig = go.Figure()

node_positions = {}

for node, position in pos.items():
    x, y = position  

    color = G.nodes[node]['color']  
    hover = G.nodes[node]['hover']
    hover = '<br>'.join(textwrap.wrap(hover,width=50))
    try:
        emotion = G.nodes[node]['emotion']
    except KeyError:
        try:
            gender = G.nodes[node]['gender']
        except KeyError:
            continue
    node_type = G.nodes[node]['type']

    if node_type == 'utterance':
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=emotion, hovertemplate=hover))
    else:
        fig.add_trace(go.Scatter(x=[x], y=[y], marker=dict(size=10,color=color), hovertext=hover, name=node_type, hovertemplate=hover))


for edge in G.edges():
    color = G.edges[edge]['color']
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines', line=dict(width=1,color=color), name='Edge'))

fig.update_layout(showlegend=False, title='Emotions Graph')

fig.update_layout(xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))

fig.show()

In [403]:
# try spektral

adjacency_matrix = nx.adjacency_matrix(G)  # Adjacency matrix
features_text = np.array([G.nodes[node]['text_embedding'] for node in G.nodes()])  # 'text' features
features_video = np.array([G.nodes[node]['visual_embedding'] for node in G.nodes()])  # 'video' features

adj_normalized = normalized_laplacian(adjacency_matrix)



In [404]:
y_train = [G.nodes[node]['emotion'] for node in G.nodes if G.nodes[node]['type'] == 'emotion']

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

y_train = ohe.fit_transform(np.array(y_train).reshape(-1,1))

y_train = tf.constant(y_train.toarray())

In [420]:
text_in = Input(shape=(features_text.shape[1],))
video_in = Input(shape=(features_video.shape[1],))
adj_in = Input((adjacency_matrix.shape[0],), sparse=True)

In [421]:

gcn1 = GCNConv(16, activation='relu')([text_in, adj_in])
gcn1 = Dropout(0.5)(gcn1)
gcn2 = GCNConv(8, activation='relu')([gcn1, adj_in])

In [422]:
merged_features = Concatenate()([gcn2, video_in])

In [423]:
fc = Dense(16, activation='relu')(merged_features)
output = Dense(len(y_train), activation='softmax')(fc)

In [424]:
model = Model(inputs=[text_in, video_in, adj_in], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [426]:
print(features_text.shape)
print(features_video.shape)
print(y_train.shape)
print(adj_normalized.shape)

(34, 50, 50)
(34, 1000)
(5, 5)
(50, 50)


In [419]:
padded_matrix = np.zeros((50, 50))
padded_matrix[:34, :34] = adj_normalized.toarray()
padded_matrix

(50, 50)

In [411]:
model.summary()

Model: "model_29"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_46 (InputLayer)          [(None, 50)]         0           []                               
                                                                                                  
 input_48 (InputLayer)          [(None, 34)]         0           []                               
                                                                                                  
 gcn_conv_23 (GCNConv)          (None, 16)           816         ['input_46[0][0]',               
                                                                  'input_48[0][0]']               
                                                                                                  
 dropout_23 (Dropout)           (None, 16)           0           ['gcn_conv_23[0][0]']     

In [413]:
y_train_pad = np.pad(y_train,(29,0))
y_train_pad.shape

(34, 34)

In [427]:
history = model.fit([features_text, features_video, adj_normalized],
          y_train_pad,
          epochs=100,
          batch_size=32,
          )

ValueError: Data cardinality is ambiguous:
  x sizes: 34, 34, 50
  y sizes: 34
Make sure all arrays contain the same number of samples.